In [175]:
import pandas as pd

# Dispatch AsyncAction

In [186]:
with open("example_lengthscales", "r") as f:
    lengthScales = pd.read_csv(f, index_col=0)

lengthScales.iloc[0:200, :]

,acure_bl_nuc,acure_ait_width,acure_cloud_ph,acure_carb_bb_diam,acure_prim_so4_diam,acure_sea_spray,acure_anth_so2_r,acure_bvoc_soa,acure_dms,acure_dry_dep_ait,...,acure_dry_dep_so2,acure_bc_ri,bparam,acure_autoconv_exp_nd,dbsdtbs_turb_0,a_ent_1_rp,modelId,latitude,longitude,time
0,9.234966,90150.421480,100000.000000,70727.856777,5.971368,6.058915,28.106973,11.358908,13.110471,12.040487,...,24.493311,11.600692,62536.398087,77321.308354,41.337929,21.463313,4754e131-3c20-4021-8fbc-c11dc2416aed,-0.625,-0.9375,2017-07-01 00:20:00
1,8.211904,35.678256,37.606812,59181.795893,4.169383,3.601145,20.367561,8.607423,8.370230,7.242221,...,10.750657,6.997130,48318.609922,21929.794673,23.903261,18.966433,86e4270f-807f-4be7-bb37-5e17a714909c,-0.625,-10.3125,2017-07-01 00:20:00
2,13.361981,1558.406016,4546.974095,8671.808673,4.730450,5.287885,21.502342,11.532419,9.704177,12.043871,...,14.259820,9.094130,11284.128104,20203.913929,36.863387,22.932885,649f4ad9-1b89-4989-8926-050c8c5212da,-0.625,-12.1875,2017-07-01 00:20:00
3,14.466849,100000.000000,43.440314,100000.000000,4.543254,4.342699,16.724894,10.026155,8.897938,14.040187,...,7.884940,9.486309,100000.000000,23334.067158,29.699939,22.069741,00133c9f-2b06-4570-a2fe-f72701b4c50e,-0.625,-14.0625,2017-07-01 00:20:00
4,15.869357,1584.367824,104.214273,3834.995025,7.352080,5.946642,30.067845,14.461436,24.957911,17.549539,...,14.610897,10.992800,3772.697973,72.868843,47.798142,35.024705,bfe42ee1-6193-4f30-9802-f4001e40bdcb,-0.625,-15.9375,2017-07-01 00:20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2.035816,15669.339458,32956.540175,3.407681,7.321065,6639.508827,3339.847198,1.060851,9294.785865,9.119272,...,8.667316,4.227666,3.414354,8.927337,2.847042,7.271916,435787e5-d729-476e-8c91-447de0ee6ca7,1.875,19.6875,2017-07-01 00:20:00
196,5.148962,31422.961038,12427.978428,4.998139,1.709623,4.826465,5.213176,3.203703,77585.486976,7751.545194,...,2.720671,5.111311,50633.275004,8215.307919,27009.257162,3.952676,3acd08e3-cb70-46f5-b4b3-34f2ea2148f9,1.875,2.8125,2017-07-01 00:20:00
197,14.792673,7.061811,8.281868,1.611201,2.607989,65602.078511,39755.993150,1.452258,1.105139,5.656720,...,4.045277,6.658243,4.451252,2.121187,5.153419,11.383761,ad087293-06ad-4d42-af4f-8ebec2602f69,1.875,21.5625,2017-07-01 00:20:00
198,10856.221475,1.833554,4.646541,2786.912710,2.129710,1648.072765,10.385432,1.128640,1.588766,1595.792065,...,7.727987,36.837667,15.683030,8.138077,390.479248,7.467555,9f46e79c-6a85-4c14-a22f-f269ebbbae06,1.875,23.4375,2017-07-01 00:20:00


In [187]:
model_ids = {}
actions = {}

for subset in range(int(200 / 30)):
    model_ids[subset] = list(lengthScales.modelId[30*subset:30*(subset+1)+1])
    actions[subset] = None

In [202]:
for mlist in model_ids.values():
    print(len(mlist))

31
31
31
31
31
31


In [189]:
kernelLen = 20
GPR_kernel = c3.SklearnGPRKernelMatern(lengthScale=[1.0]*kernelLen, nu=0.5, coefficient=1.0).build().kernel.upsert()

GPR_technique = c3.GaussianProcessRegressionTechnique(
                    randomState=42,
                    kernel = GPR_kernel
).upsert()

GPR_dataspec = c3.GPRDataSourceSpec(
).upsert()

In [190]:
for idx in model_ids.keys():

    GPR_pipe = c3.GaussianProcessRegressionPipe(
        technique=GPR_technique,
        id="trial" + str(idx)
    )

    # action spec
    spec = c3.AsyncActionSpec(
        typeName="GaussianProcessRegressionPipe",
        action="trainWithListOfAODModels",
        args={
            "modelIds": model_ids[idx],
            "excludeFeatures": [],
            "this": GPR_pipe.toJson()
        }
    )
    
    action = c3.AsyncAction.submit(spec)
    actions[idx] = action

In [191]:
actions

{0: c3.AsyncAction(
 typeName='GaussianProcessRegressionPipe',
 actionName='trainWithListOfAODModels',
 args=c3.Mapp<string, any>({'excludeFeatures': [],
        'modelIds': ['4754e131-3c20-4021-8fbc-c11dc2416aed',
                     '86e4270f-807f-4be7-bb37-5e17a714909c',
                     '649f4ad9-1b89-4989-8926-050c8c5212da',
                     '00133c9f-2b06-4570-a2fe-f72701b4c50e',
                     'bfe42ee1-6193-4f30-9802-f4001e40bdcb',
                     'ec18e811-34a5-4cf4-8242-dd671427772f',
                     '0f842fea-853e-4fc8-9fc8-fe08f57fa6b8',
                     '0328811a-496d-449a-86e3-342c3625738f',
                     '593a1a98-261a-40be-a1e7-956ef8264b2d',
                     '5dcfab0c-0109-4183-84ab-4b384eba66ca',
                     '38cf28d0-f527-4289-afdc-8b4819125169',
                     '05a92fe6-63c0-4fba-96e6-806ada179149',
                     '4e8527b7-e9ec-4df4-89f9-e3acf0acb6a2',
                     '0589f2f8-1069-4068-a4b7-99eb481

In [200]:
for action in actions.values():
    print(action.hasCompleted())

False
True
True
False
True
False


### After complete, check to see if model was trained

In [136]:
for idx in model_ids.keys():
    trained = c3.GaussianProcessRegressionPipe.get("trial" + str(idx))
    print(trained)

In [205]:
c3.ActionRef.get('70c906db-f5bf-4145-b18b-42a0a20d4a6b')

AttributeError: Type "ActionRef" has no attribute "get"